## Podział danych na dwa foldery. Folder zawierający dane testowe około 10% i folder zawieracjący dane treningowe około 90%

#### Importowanie bibliotek

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

#### Funkcja `extract_lesion_df`
Funkcja odpowiedzialna za wyodrębnienie unikalnych lesion_id i ich grup ryzyka

In [2]:
def extract_lesion_df(df):
    return (df[["lesion_id", "risk_group"]].drop_duplicates(subset="lesion_id").reset_index(drop=True))

#### Funkcja `split_lesions`
Funkcja odpowiedzialna za podzielenie danych z uwzględnieniem stratify

In [3]:
def split_lesions(lesion_df, test_size = 0.10, random_state = 42):
    train_le, test_le = train_test_split(lesion_df, test_size=test_size, random_state=random_state, stratify=lesion_df["risk_group"])
    return train_le["lesion_id"].tolist(), test_le["lesion_id"].tolist()

#### Funkcja `filter_by_ids`
Funkcja odpowiedzialna za filtrowanie wierszy na podstawie listy identyfikatorów

In [4]:
def filter_by_ids(df, ids, id_field = "lesion_id"):
    return df[df[id_field].isin(ids)].reset_index(drop=True)

#### Funkcja `save_splits`
Funkcja odpowiedzialna za zapisanie przeprowadzonego podziału do plików .csv

In [5]:
def save_splits(train_df, test_df, output_dir):
    output_dir.mkdir(parents=True, exist_ok=True)
    train_df.to_csv(output_dir / "train.csv", index=False)
    test_df.to_csv(output_dir / "test.csv", index=False)
    print(f"Zapisano train.csv i test.csv w {output_dir}")
    print(f"Wierszy: train={len(train_df)}, test={len(test_df)}")

#### Funkcja `run_data_split`
Funckja składająca wywołanie wymaganych funkcji do przeprowadznia podziału danych

In [6]:
def run_data_split(data_csv, output_dir, test_size= 0.10, random_state = 42):
    df = pd.read_csv(data_csv)
    lesion_df = extract_lesion_df(df)
    train_ids, test_ids = split_lesions(lesion_df, test_size, random_state)

    train_df = filter_by_ids(df, train_ids)
    test_df  = filter_by_ids(df, test_ids)

    save_splits(train_df, test_df, output_dir)